# 秘密鍵から公開鍵、アドレスを生成する

In [10]:
import os
import ecdsa
import hashlib
import base58

private_key = os.urandom(32)
public_key = ecdsa.SigningKey.from_string(private_key,curve=ecdsa.SECP256k1).verifying_key.to_string()

curve : 楕円曲線を指定

In [11]:
prefix_and_pubkey = b"\x04" + public_key

b"\x04"はバイナリ,public_keyの前に非圧縮公開鍵のプレフィックスとしてくっつける

In [13]:
intermediate = hashlib.sha256(prefix_and_pubkey).digest()
ripemd160 = hashlib.new("ripemd160")
ripemd160.update(intermediate)
hash160 = ripemd160.digest()

SHA-256でハッシュし、さらにRIPEMD-160でハッシュ(この過程をまとめて**HASH160**と呼ぶ)

In [14]:
prefix_and_hash160 = b"\x00" + hash160

公開鍵ハッシュに、公開鍵ハッシュのバージョンプレフィクスである00をくっつける　バージョンプレフィックスとは、生成するアドレスがどのようなタイプのアドレスなのかを示す16進数の情報である

In [15]:
double_hash = hashlib.sha256(hashlib.sha256(prefix_and_hash160).digest()).digest()
checksum = double_hash[:4]

SHA-256で2回ハッシュした後、先頭4バイトを取り出す

In [16]:
pre_address = prefix_and_hash160 + checksum
address = base58.b58encode(pre_address)
print(address.decode())

1DpcbxCbksr1fF1sPL1ipEZKa6aRR4EHTg


バージョンプレフィックス+公開鍵のハッシュ値+チェックサムをBase58エンコードすることでアドレスを生成

![how_to_create](how_to_create.png)
拾い画